Pacote threading:

https://youtu.be/A_Z1lgZLSNc?si=6RaFKqEfAT8aLO_2

https://youtu.be/dkptw_R7Qhc?si=oUSejtFv9gYvDNX3

Paocte pygetwindow:

https://pypi.org/project/PyGetWindow/

**Código para rodar no cluster**

In [ ]:
import random
import time
import threading
import pygetwindow as gw
from itertools import combinations
from simnibs import sim_struct, run_simnibs

# INICIALIZANDO A SESSÃO
s = sim_struct.SESSION()

# NOME DA MALHA DA CABEÇA
s.subpath = r'C:\Users\Ranulfo\Documents\simnibs4_examples\m2m_ernie'

# PASTA DE SAÍDA
s.pathfem = 'tutorial100/'

# LISTA DE POSIÇÕES
safe_positions = ['AF3', 'AF4', 'AF7', 'AF8', 'AFz', 'C1', 'C2', 'C3', 'C4', 'C5', 'C6', 'CP1', 'CP2', 'CP3', 'CP4', 'CP5',
                  'CP6', 'CPz', 'Cz', 'F1', 'F10', 'F2', 'F3', 'F4',
                  'F5', 'F6', 'F7', 'F8', 'F9', 'FC1', 'FC2', 'FC3', 'FC4', 'FC5', 'FC6', 'FCz', 'FT10', 'FT7', 'FT8',
                  'FT9', 'Fp1', 'Fp2', 'Fpz', 'Fz', 'I1', 'I2', 'Iz', 'LPA', 'Nz',
                  'P1', 'P10', 'P2', 'P3', 'P4', 'P5', 'P6', 'P7', 'P8', 'P9', 'PO10', 'PO3', 'PO4', 'PO7', 'PO8', 'PO9',
                  'POz', 'Pz', 'RPA', 'T10', 'T7',
                  'T8', 'T9', 'TP10', 'TP7', 'TP8', 'TP9']

# GERANDO TODAS AS COMBINAÇÕES POSSÍVEIS DE 4 ELETRODOS
electrode_channels = list(range(1, 65))  # Assume 64 canais de eletrodos possíveis
valid_electrode_combinations_4 = list(combinations(electrode_channels, 4))

# NÚMERO DE SIMULAÇÕES PARA 4 ELETRODOS
num_simulations_4 = len(valid_electrode_combinations_4)

simulations_data = []

# LOOP DE SIMULAÇÃO PARA 4 ELETRODOS
for sim_num in range(num_simulations_4):

    # OBTENHA A COMBINAÇÃO DE ELETRODOS ATUAL
    electrodes = valid_electrode_combinations_4[sim_num]

    # SELECIONANDO 2 CÁTODOS E 2 ÂNODOS ALEATORIAMENTE
    selected_cathodes = random.sample(electrodes, 2)
    selected_anodes = random.sample([ch for ch in electrodes if ch not in selected_cathodes], 2)

    # INICIALIZAÇÃO DA SIMULAÇÃO tDCS
    tdcslist = s.add_tdcslist()
    tdcslist.currents = [-1e-3, 1e-3, -1e-3, 1e-3]

    # INICIALIZAÇÃO DOS CÁTODOS
    for cathode_ch in selected_cathodes:
        cathode = tdcslist.add_electrode()
        cathode.channelnr = cathode_ch
        cathode.dimensions = [30]  # Define o diâmetro do eletrodo cátodo como 30 mm
        cathode.shape = 'ellipse'  # Define a forma do eletrodo cátodo como circular
        cathode.thickness = 5
        cathode.centre = random.choice(safe_positions)

    # INICIALIZAÇÃO DOS ÂNODOS
    for anode_ch in selected_anodes:
        anode = tdcslist.add_electrode()
        anode.channelnr = anode_ch
        anode.dimensions = [30]  # Define o diâmetro do eletrodo ânodo como 30 mm
        anode.shape = 'ellipse'  # Define a forma do eletrodo ânodo como circular
        anode.thickness = 5

        # DEFINA A POSIÇÃO DO ÂNODO DIFERENTE DA POSIÇÃO DO CÁTODO
        other_positions = [pos for pos in safe_positions if pos not in selected_cathodes]
        anode.centre = random.choice(other_positions)

# CLASSE PARA FECHAR AS JANELAS .msh
def close_all_msh_windows():
    while True:
        for window in gw.getAllTitles():
            if window.endswith(".msh"):
                window_to_close = gw.getWindowsWithTitle(window)
                if window_to_close:
                    window_to_close[0].close()
        time.sleep(1)  #Fecha em um segundo

# RODAR A SIMULAÇÃO
def run_simulation(s):
    run_simnibs(s)

# INICIA PARA RODAR AS SIMULAÇÕES
simulation_thread = threading.Thread(target=run_simulation, args=(s,))
simulation_thread.start()

# FECHA AS JANELAS .msh
window_closing_thread = threading.Thread(target=close_all_msh_windows)
window_closing_thread.start()

# COMANDO PARA AGUARDAR O PROCESSO
simulation_thread.join()

# AO TERMINAR OS PROCESSOS DA SIMULAÇÃO É FINALIZADO A THREAD
window_closing_thread.join()

**Código de entrada**

In [ ]:
import random
import time
import threading
import pygetwindow as gw
from itertools import combinations
from simnibs import sim_struct, run_simnibs

# INICIALIZANDO A SESSÃO
s = sim_struct.SESSION()

# NOME DA MALHA DA CABEÇA
s.subpath = r'C:\Users\Ranulfo\Documents\simnibs4_examples\m2m_ernie'

# PASTA DE SAÍDA
s.pathfem = 'tutorial5/'

# ENTRADA DO USUÁRIO PARA COORDENADAS DO TUMOR
tumor_x = float(input("Digite a coordenada x onde será gerado o campo elétrico: "))
tumor_y = float(input("Digite a coordenada y onde será gerado o campo elétrico: "))
tumor_z = float(input("Digite a coordenada z onde será gerado o campo elétrico: "))

# Lista de posições seguras dos eletrodos
safe_positions = ['AF3', 'AF4', 'AF7', 'AF8', 'AFz', 'C1', 'C2', 'C3', 'C4', 'C5', 'C6', 'CP1', 'CP2', 'CP3', 'CP4', 'CP5',
                  'CP6', 'CPz', 'Cz', 'F1', 'F10', 'F2', 'F3', 'F4', 'F5', 'F6', 'F7', 'F8', 'F9', 'FC1', 'FC2', 'FC3',
                  'FC4', 'FC5', 'FC6', 'FCz', 'FT10', 'FT7', 'FT8', 'FT9', 'Fp1', 'Fp2', 'Fpz', 'Fz', 'I1', 'I2', 'Iz',
                  'LPA', 'Nz', 'P1', 'P10', 'P2', 'P3', 'P4', 'P5', 'P6', 'P7', 'P8', 'P9', 'PO10', 'PO3', 'PO4', 'PO7',
                  'PO8', 'PO9', 'POz', 'Pz', 'RPA', 'T10', 'T7', 'T8', 'T9', 'TP10', 'TP7', 'TP8', 'TP9']

# Gerando lista de canais dos eletrodos para 4 eletrodos
electrode_channels_4 = [1, 2, 3, 4]
valid_electrode_combinations_4 = list(combinations(electrode_channels_4, 4))

# Número de simulações para 4 eletrodos
num_simulations_4 = 1

simulations_data = []

# Loop de simulação para 4 eletrodos
for sim_num in range(num_simulations_4):
    # Selecionando uma combinação de eletrodo
    electrodes = random.choice(valid_electrode_combinations_4)

    # Selecionando 2 cátodos e 2 ânodos aleatoriamente
    selected_cathodes = random.sample(electrodes, 2)
    selected_anodes = random.sample([ch for ch in electrodes if ch not in selected_cathodes], 2)

    # Inicialização da simulação tDCS
    tdcslist = s.add_tdcslist()

    # Adicionando os parâmetros de corrente alternada
    frequency = 1  # Frequência em Hertz
    amplitude = 1e-3  # Amplitude da corrente em Ampères
    tdcslist.currents = [amplitude * (2 if sim_num % 2 == 0 else -2) * (1 if i % 2 == 0 else -1) for i in range(4)]
    tdcslist.frequency = frequency

    # Inicialização dos cátodos
    for cathode_ch in selected_cathodes:
        cathode = tdcslist.add_electrode()
        cathode.channelnr = cathode_ch
        cathode.dimensions = [30]  # Define o diâmetro do eletrodo cátodo como 30 mm
        cathode.shape = 'ellipse'  # Define a forma do eletrodo cátodo como circular
        cathode.thickness = 5
        cathode.centre = random.choice(safe_positions)

    # Inicialização dos ânodos
    for anode_ch in selected_anodes:
        anode = tdcslist.add_electrode()
        anode.channelnr = anode_ch
        anode.dimensions = [30]  # Define o diâmetro do eletrodo ânodo como 30 mm
        anode.shape = 'ellipse'  # Define a forma do eletrodo ânodo como circular
        anode.thickness = 5

        # Define a posição do ânodo diferente da posição do cátodo
        other_positions = [pos for pos in safe_positions if pos not in selected_cathodes]
        anode.centre = random.choice(other_positions)

    # Rodar a simulação
    run_simnibs(s)

# Classe para fechar as janelas .msh
def close_all_msh_windows():
    while True:
        for window in gw.getAllTitles():
            if window.endswith(".msh"):
                window_to_close = gw.getWindowsWithTitle(window)
                if window_to_close:
                    window_to_close[0].close()
        time.sleep(1)  # Fecha em um segundo

# Iniciar a thread para fechar as janelas .msh
window_closing_thread = threading.Thread(target=close_all_msh_windows)
window_closing_thread.start()

# Aguardar o término do processo
window_closing_thread.join()


In [ ]:
import random
import time
import threading
import pygetwindow as gw
from itertools import combinations
from simnibs import sim_struct, run_simnibs

# INICIALIZANDO A SESSÃO
s = sim_struct.SESSION()

# NOME DA MALHA DA CABEÇA
s.subpath = r'C:\Users\Ranulfo\Documents\simnibs4_examples\m2m_ernie'

# PASTA DE SAÍDA
s.pathfem = 'tutorial3/'

# ENTRADA DO USUÁRIO PARA COORDENADAS DO TUMOR
tumor_x = float(input("Digite a coordenada x onde será gerado o campo elétrico: "))
tumor_y = float(input("Digite a coordenada y onde será gerado o campo elétrico: "))
tumor_z = float(input("Digite a coordenada z onde será gerado o campo elétrico: "))

# ENTRADA DO USUÁRIO PARA O VOLUME DO TUMOR
tumor_volume = float(input("Digite o volume do tumor (em mm^3): "))

# Calculate the target volume for the electric field (10% larger than the tumor volume)
target_volume = 1.1 * tumor_volume

# LISTA DE POSIÇÕES
safe_positions = ['AF3', 'AF4', 'AF7', 'AF8', 'AFz', 'C1', 'C2', 'C3', 'C4', 'C5', 'C6', 'CP1', 'CP2', 'CP3', 'CP4', 'CP5',
                  'CP6', 'CPz', 'Cz', 'F1', 'F10', 'F2', 'F3', 'F4', 'F5', 'F6', 'F7', 'F8', 'F9', 'FC1', 'FC2', 'FC3',
                  'FC4', 'FC5', 'FC6', 'FCz', 'FT10', 'FT7', 'FT8', 'FT9', 'Fp1', 'Fp2', 'Fpz', 'Fz', 'I1', 'I2', 'Iz',
                  'LPA', 'Nz', 'P1', 'P10', 'P2', 'P3', 'P4', 'P5', 'P6', 'P7', 'P8', 'P9', 'PO10', 'PO3', 'PO4', 'PO7',
                  'PO8', 'PO9', 'POz', 'Pz', 'RPA', 'T10', 'T7', 'T8', 'T9', 'TP10', 'TP7', 'TP8', 'TP9']

# GERANDO LISTA DE CANAIS DOS ELETRODOS PARA 4 ELETRODOS
electrode_channels_4 = [1, 2, 3, 4]
valid_electrode_combinations_4 = list(combinations(electrode_channels_4, 4))

# NÚMERO DE SIMULAÇÕES PARA 4 ELETRODOS
num_simulations_4 = 1

simulations_data = []

# LOOP DE SIMULAÇÃO PARA 4 ELETRODOS
for sim_num in range(num_simulations_4):
    # SELECIONE UMA COMBINAÇÃO DE ELETRODO
    electrodes = random.choice(valid_electrode_combinations_4)

    # SELECIONANDO 2 CÁTODOS E 2 ÂNODOS ALEATORIAMENTE
    selected_cathodes = random.sample(electrodes, 2)
    selected_anodes = random.sample([ch for ch in electrodes if ch not in selected_cathodes], 2)

    # INICIALIZAÇÃO DA SIMULAÇÃO tDCS
    tdcslist = s.add_tdcslist()

    # Adicione os parâmetros de corrente alternada
    frequency = 1  # Frequência em Hertz
    amplitude = 1e-3  # Amplitude da corrente em Ampères
    tdcslist.currents = [amplitude * (2 if sim_num % 2 == 0 else -2) * (1 if i % 2 == 0 else -1) for i in range(4)]
    tdcslist.frequency = frequency

    # INICIALIZAÇÃO DOS CÁTODOS
    for cathode_ch in selected_cathodes:
        cathode = tdcslist.add_electrode()
        cathode.channelnr = cathode_ch
        cathode.dimensions = [30]  # Define o diâmetro do eletrodo cátodo como 30 mm
        cathode.shape = 'ellipse'  # Define a forma do eletrodo cátodo como circular
        cathode.thickness = 5
        cathode.centre = random.choice(safe_positions)

    # INICIALIZAÇÃO DOS ÂNODOS
    for anode_ch in selected_anodes:
        anode = tdcslist.add_electrode()
        anode.channelnr = anode_ch
        anode.dimensions = [30]  # Define o diâmetro do eletrodo ânodo como 30 mm
        anode.shape = 'ellipse'  # Define a forma do eletrodo ânodo como circular
        anode.thickness = 5

        # DEFINA A POSIÇÃO DO ÂNODO DIFERENTE DA POSIÇÃO DO CÁTODO
        other_positions = [pos for pos in safe_positions if pos not in selected_cathodes]
        anode.centre = random.choice(other_positions)

    # Check if the simulated field volume meets the criteria
    if tdcslist.volume > target_volume:
        simulations_data.append((sim_num, tdcslist.volume, tdcslist))
        break  # Exit the loop if a suitable simulation is found
    else:
        # If the volume is not large enough, remove the current tdcslist
        s.tdcslist.remove(tdcslist)

# CLASSE PARA FECHAR AS JANELAS .msh
def close_all_msh_windows():
    while True:
        for window in gw.getAllTitles():
            if window.endswith(".msh"):
                window_to_close = gw.getWindowsWithTitle(window)
                if window_to_close:
                    window_to_close[0].close()
        time.sleep(1)  # Fecha em um segundo

# RODAR A SIMULAÇÃO
def run_simulation(s):
    run_simnibs(s)

# INICIA PARA RODAR AS SIMULAÇÕES
simulation_thread = threading.Thread(target=run_simulation, args=(s,))
simulation_thread.start()

# FECHA AS JANELAS .msh
window_closing_thread = threading.Thread(target=close_all_msh_windows)
window_closing_thread.start()

# COMANDO PARA AGUARDAR O PROCESSO
simulation_thread.join()

# AO TERMINAR OS PROCESSOS DA SIMULAÇÃO É FINALIZADO A THREAD
window_closing_thread.join()


**Código corrente contínua ETCC**

**4 eletrodos**

In [ ]:
import random
import time
import threading
import pygetwindow as gw
from itertools import combinations
from simnibs import sim_struct, run_simnibs

# INICIALIZANDO A SESSÃO
s = sim_struct.SESSION()

# NOME DA MALHA DA CABEÇA
s.subpath =  r'C:\Users\Ranulfo\Documents\simnibs4_examples\m2m_ernie'

# PASTA DE SAÍDA
s.pathfem = 'tutorial100/'

# LISTA DE POSIÇÕES
safe_positions = ['AF3', 'AF4', 'AF7', 'AF8', 'AFz', 'C1', 'C2', 'C3', 'C4', 'C5', 'C6', 'CP1', 'CP2', 'CP3', 'CP4', 'CP5', 'CP6', 'CPz', 'Cz', 'F1', 'F10', 'F2', 'F3', 'F4',
'F5', 'F6', 'F7', 'F8', 'F9', 'FC1', 'FC2', 'FC3', 'FC4', 'FC5', 'FC6', 'FCz', 'FT10', 'FT7', 'FT8', 'FT9', 'Fp1', 'Fp2', 'Fpz', 'Fz', 'I1', 'I2', 'Iz', 'LPA', 'Nz',
'P1', 'P10', 'P2', 'P3', 'P4', 'P5', 'P6', 'P7', 'P8', 'P9', 'PO10', 'PO3', 'PO4', 'PO7', 'PO8', 'PO9', 'POz', 'Pz', 'RPA', 'T10', 'T7',
'T8', 'T9', 'TP10', 'TP7', 'TP8', 'TP9']

# GERANDO LISTA DE CANAIS DOS ELETRODOS PARA 4 ELETRODOS
electrode_channels_4 = [1, 2, 3, 4]
valid_electrode_combinations_4 = list(combinations(electrode_channels_4, 4))

# NÚMERO DE SIMULAÇÕES PARA 4 ELETRODOS
num_simulations_4 = 2

simulations_data = []

# LOOP DE SIMULAÇÃO PARA 4 ELETRODOS
for sim_num in range(num_simulations_4):

    # SELECIONE UMA COMBINAÇÃO DE ELETRODO
    electrodes = random.choice(valid_electrode_combinations_4)

    # SELECIONANDO 2 CÁTODOS E 2 ÂNODOS ALEATORIAMENTE
    selected_cathodes = random.sample(electrodes, 2)
    selected_anodes = random.sample([ch for ch in electrodes if ch not in selected_cathodes], 2)

    # INICIALIZAÇÃO DA SIMULAÇÃO tDCS
    tdcslist = s.add_tdcslist()
    tdcslist.currents = [-1e-3, 1e-3, -1e-3, 1e-3]

    # INICIALIZAÇÃO DOS CÁTODOS
    for cathode_ch in selected_cathodes:
        cathode = tdcslist.add_electrode()
        cathode.channelnr = cathode_ch
        cathode.dimensions = [30]  # Define o diâmetro do eletrodo cátodo como 30 mm
        cathode.shape = 'ellipse'  # Define a forma do eletrodo cátodo como circular
        cathode.thickness = 5
        cathode.centre = random.choice(safe_positions)

    # INICIALIZAÇÃO DOS ÂNODOS
    for anode_ch in selected_anodes:
        anode = tdcslist.add_electrode()
        anode.channelnr = anode_ch
        anode.dimensions = [30]    # Define o diâmetro do eletrodo ânodo como 30 mm
        anode.shape = 'ellipse'    # Define a forma do eletrodo ânodo como circular
        anode.thickness = 5

        # DEFINA A POSIÇÃO DO ÂNODO DIFERENTE DA POSIÇÃO DO CÁTODO
        other_positions = [pos for pos in safe_positions if pos not in selected_cathodes]
        anode.centre = random.choice(other_positions)


# CLASSE PARA FECHAR AS JANELAS .msh
def close_all_msh_windows():
    while True:
        for window in gw.getAllTitles():
            if window.endswith(".msh"):
                window_to_close = gw.getWindowsWithTitle(window)
                if window_to_close:
                    window_to_close[0].close()
        time.sleep(1)  #Fecha em um segundo

# RODAR A SIMULAÇÃO
def run_simulation(s):
    run_simnibs(s)

# INICIA PARA RODAR AS SIMULAÇÕES
simulation_thread = threading.Thread(target=run_simulation, args=(s,))
simulation_thread.start()

# FECHA AS JANELAS .msh
window_closing_thread = threading.Thread(target=close_all_msh_windows)
window_closing_thread.start()

# COMANDO PARA AGUARDAR O PROCESSO
simulation_thread.join()

# AO TERMINAR OS PROCESSOS DA SIMULAÇÃO É FINALIZADO A THREAD
window_closing_thread.join()


**6 eletrodos**

In [ ]:
import random
import time
import threading
import pygetwindow as gw
from itertools import combinations
from simnibs import sim_struct, run_simnibs

# INICIALIZANDO A SESSÃO
s = sim_struct.SESSION()

# NOME DA MALHA DA CABEÇA
s.subpath =  r'C:\Users\Ranulfo\Documents\simnibs4_examples\m2m_ernie'

# PASTA DE SAÍDA
s.pathfem = 'tutorial98/'

# LISTA DE POSIÇÕES
safe_positions = ['AF3', 'AF4', 'AF7', 'AF8', 'AFz', 'C1', 'C2', 'C3', 'C4', 'C5', 'C6', 'CP1', 'CP2', 'CP3', 'CP4', 'CP5', 'CP6', 'CPz', 'Cz', 'F1', 'F10', 'F2', 'F3', 'F4',
'F5', 'F6', 'F7', 'F8', 'F9', 'FC1', 'FC2', 'FC3', 'FC4', 'FC5', 'FC6', 'FCz', 'FT10', 'FT7', 'FT8', 'FT9', 'Fp1', 'Fp2', 'Fpz', 'Fz', 'I1', 'I2', 'Iz', 'LPA', 'Nz',
'P1', 'P10', 'P2', 'P3', 'P4', 'P5', 'P6', 'P7', 'P8', 'P9', 'PO10', 'PO3', 'PO4', 'PO7', 'PO8', 'PO9', 'POz', 'Pz', 'RPA', 'T10', 'T7',
'T8', 'T9', 'TP10', 'TP7', 'TP8', 'TP9']

# GERANDO LISTA DE CANAIS DOS ELETRODOS PARA 6 ELETRODOS
electrode_channels_6 = [1, 2, 3, 4, 5, 6]
valid_electrode_combinations_6 = list(combinations(electrode_channels_6, 6))

# NÚMERO DE SIMULAÇÕES PARA 6 ELETRODOS
num_simulations_6 = 170

simulations_data = []

# LOOP DE SIMULAÇÃO PARA 6 ELETRODOS
for sim_num in range(num_simulations_6):

    # SELECIONE UMA COMBINAÇÃO DE ELETRODO
    electrodes = random.choice(valid_electrode_combinations_6)

    # SELECIONANDO 3 CÁTODOS E 3 ÂNODOS ALEATORIAMENTE
    selected_cathodes = random.sample(electrodes, 3)
    selected_anodes = random.sample([ch for ch in electrodes if ch not in selected_cathodes], 3)

    # INICIALIZAÇÃO DA SIMULAÇÃO tDCS
    tdcslist = s.add_tdcslist()
    tdcslist.currents = [-1e-3, 1e-3, -1e-3, 1e-3, -1e-3, 1e-3]

    # INICIALIZAÇÃO DOS CÁTODOS
    for cathode_ch in selected_cathodes:
        cathode = tdcslist.add_electrode()
        cathode.channelnr = cathode_ch
        cathode.dimensions = [30]  # Define o diâmetro do eletrodo cátodo como 30 mm
        cathode.shape = 'ellipse'  # Define a forma do eletrodo cátodo como circular
        cathode.thickness = 5
        cathode.centre = random.choice(safe_positions)

    # INICIALIZAÇÃO DOS ÂNODOS
    for anode_ch in selected_anodes:
        anode = tdcslist.add_electrode()
        anode.channelnr = anode_ch
        anode.dimensions = [30]    # Define o diâmetro do eletrodo ânodo como 30 mm
        anode.shape = 'ellipse'    # Define a forma do eletrodo ânodo como circular
        anode.thickness = 5

        # DEFINA A POSIÇÃO DO ÂNODO DIFERENTE DA POSIÇÃO DO CÁTODO
        other_positions = [pos for pos in safe_positions if pos not in selected_cathodes]
        anode.centre = random.choice(other_positions)

# CLASSE PARA FECHAR AS JANELAS .msh
def close_all_msh_windows():
    while True:
        for window in gw.getAllTitles():
            if window.endswith(".msh"):
                window_to_close = gw.getWindowsWithTitle(window)
                if window_to_close:
                    window_to_close[0].close()
        time.sleep(1)  #Fecha em um segundo

# RODAR A SIMULAÇÃO
def run_simulation(s):
    run_simnibs(s)

# INICIA PARA RODAR AS SIMULAÇÕES
simulation_thread = threading.Thread(target=run_simulation, args=(s,))
simulation_thread.start()

# FECHA AS JANELAS .msh
window_closing_thread = threading.Thread(target=close_all_msh_windows)
window_closing_thread.start()

# COMANDO PARA AGUARDAR O PROCESSO
simulation_thread.join()

# AO TERMINAR OS PROCESSOS DA SIMULAÇÃO É FINALIZADO A THREAD
window_closing_thread.join()

**8 eletrodos**

In [ ]:
import random
import time
import threading
import pygetwindow as gw
from itertools import combinations
from simnibs import sim_struct, run_simnibs

# INICIALIZANDO A SESSÃO
s = sim_struct.SESSION()

# NOME DA MALHA DA CABEÇA
s.subpath =  r'C:\Users\Ranulfo\Documents\simnibs4_examples\m2m_ernie'

# PASTA DE SAÍDA
s.pathfem = 'tutorial99/'

# LISTA DE POSIÇÕES
safe_positions = ['AF3', 'AF4', 'AF7', 'AF8', 'AFz', 'C1', 'C2', 'C3', 'C4', 'C5', 'C6', 'CP1', 'CP2', 'CP3', 'CP4', 'CP5', 'CP6', 'CPz', 'Cz', 'F1', 'F10', 'F2', 'F3', 'F4',
'F5', 'F6', 'F7', 'F8', 'F9', 'FC1', 'FC2', 'FC3', 'FC4', 'FC5', 'FC6', 'FCz', 'FT10', 'FT7', 'FT8', 'FT9', 'Fp1', 'Fp2', 'Fpz', 'Fz', 'I1', 'I2', 'Iz', 'LPA', 'Nz',
'P1', 'P10', 'P2', 'P3', 'P4', 'P5', 'P6', 'P7', 'P8', 'P9', 'PO10', 'PO3', 'PO4', 'PO7', 'PO8', 'PO9', 'POz', 'Pz', 'RPA', 'T10', 'T7',
'T8', 'T9', 'TP10', 'TP7', 'TP8', 'TP9']

# GERANDO LISTA DE CANAIS DOS ELETRODOS PARA 8 ELETRODOS
electrode_channels_8 = [1, 2, 3, 4, 5, 6, 7, 8]
valid_electrode_combinations_8 = list(combinations(electrode_channels_8, 8))

# NÚMERO DE SIMULAÇÕES PARA 8 ELETRODOS
num_simulations_8 = 150

simulations_data = []

# LOOP DE SIMULAÇÃO PARA 8 ELETRODOS
for sim_num in range(num_simulations_8):

    # SELECIONE UMA COMBINAÇÃO DE ELETRODO
    electrodes = random.choice(valid_electrode_combinations_8)

    # SELECIONANDO 4 CÁTODOS E 4 ÂNODOS ALEATORIAMENTE
    selected_cathodes = random.sample(electrodes, 4)
    selected_anodes = random.sample([ch for ch in electrodes if ch not in selected_cathodes], 4)

    # INICIALIZAÇÃO DA SIMULAÇÃO tDCS
    tdcslist = s.add_tdcslist()
    tdcslist.currents = [-1e-3, 1e-3, -1e-3, 1e-3, -1e-3, 1e-3, -1e-3, 1e-3]

    # INICIALIZAÇÃO DOS CÁTODOS
    for cathode_ch in selected_cathodes:
        cathode = tdcslist.add_electrode()
        cathode.channelnr = cathode_ch
        cathode.dimensions = [30]  # Define o diâmetro do eletrodo cátodo como 30 mm
        cathode.shape = 'ellipse'  # Define a forma do eletrodo cátodo como circular
        cathode.thickness = 5
        cathode.centre = random.choice(safe_positions)

    # INICIALIZAÇÃO DOS ÂNODOS
    for anode_ch in selected_anodes:
        anode = tdcslist.add_electrode()
        anode.channelnr = anode_ch
        anode.dimensions = [30]    # Define o diâmetro do eletrodo ânodo como 30 mm
        anode.shape = 'ellipse'    # Define a forma do eletrodo ânodo como circular
        anode.thickness = 5

        # DEFINA A POSIÇÃO DO ÂNODO DIFERENTE DA POSIÇÃO DO CÁTODO
        other_positions = [pos for pos in safe_positions if pos not in selected_cathodes]
        anode.centre = random.choice(other_positions)

# CLASSE PARA FECHAR AS JANELAS .msh
def close_all_msh_windows():
    while True:
        for window in gw.getAllTitles():
            if window.endswith(".msh"):
                window_to_close = gw.getWindowsWithTitle(window)
                if window_to_close:
                    window_to_close[0].close()
        time.sleep(30)  #Fecha em 30 segundos

# RODAR A SIMULAÇÃO
def run_simulation(s):
    run_simnibs(s)

# INICIA PARA RODAR AS SIMULAÇÕES
simulation_thread = threading.Thread(target=run_simulation, args=(s,))
simulation_thread.start()

# FECHA AS JANELAS .msh
window_closing_thread = threading.Thread(target=close_all_msh_windows)
window_closing_thread.start()

# COMANDO PARA AGUARDAR O PROCESSO
simulation_thread.join()

# AO TERMINAR OS PROCESSOS DA SIMULAÇÃO É FINALIZADO A THREAD
window_closing_thread.join()

**Corrente Alternada**

**4 eletrodos**

In [ ]:
import random
import time
import threading
import pygetwindow as gw
from itertools import combinations
from simnibs import sim_struct, run_simnibs

# INICIALIZANDO A SESSÃO
s = sim_struct.SESSION()

# NOME DA MALHA DA CABEÇA
s.subpath = r'C:\Users\laboratorio\Documents\simnibs4_examples\m2m_ernie'

# PASTA DE SAÍDA
s.pathfem = 'tutorial4.2/'

# LISTA DE POSIÇÕES
safe_positions = ['AF3', 'AF4', 'AF7', 'AF8', 'AFz', 'C1', 'C2', 'C3', 'C4', 'C5', 'C6', 'CP1', 'CP2', 'CP3', 'CP4', 'CP5',
                   'CP6', 'CPz', 'Cz', 'F1', 'F10', 'F2', 'F3', 'F4', 'F5', 'F6', 'F7', 'F8', 'F9', 'FC1', 'FC2', 'FC3',
                   'FC4', 'FC5', 'FC6', 'FCz', 'FT10', 'FT7', 'FT8', 'FT9', 'Fp1', 'Fp2', 'Fpz', 'Fz', 'I1', 'I2', 'Iz',
                   'LPA', 'Nz', 'P1', 'P10', 'P2', 'P3', 'P4', 'P5', 'P6', 'P7', 'P8', 'P9', 'PO10', 'PO3', 'PO4', 'PO7',
                   'PO8', 'PO9', 'POz', 'Pz', 'RPA', 'T10', 'T7', 'T8', 'T9', 'TP10', 'TP7', 'TP8', 'TP9']

# GERANDO LISTA DE CANAIS DOS ELETRODOS PARA 4 ELETRODOS
electrode_channels_4 = [1, 2, 3, 4]
valid_electrode_combinations_4 = list(combinations(electrode_channels_4, 4))

# NÚMERO DE SIMULAÇÕES PARA 4 ELETRODOS
num_simulations_4 = 2

simulations_data = []

# LOOP DE SIMULAÇÃO PARA 4 ELETRODOS
for sim_num in range(num_simulations_4):

    # SELECIONE UMA COMBINAÇÃO DE ELETRODO
    electrodes = random.choice(valid_electrode_combinations_4)

    # SELECIONANDO 2 CÁTODOS E 2 ÂNODOS ALEATORIAMENTE
    selected_cathodes = random.sample(electrodes, 2)
    selected_anodes = random.sample([ch for ch in electrodes if ch not in selected_cathodes], 2)

    # INICIALIZAÇÃO DA SIMULAÇÃO tDCS
    tdcslist = s.add_tdcslist()

    # Adicione os parâmetros de corrente alternada
    frequency = 1  # Frequência em Hertz
    amplitude = 1e-3  # Amplitude da corrente em Ampères
    tdcslist.currents = [amplitude * (2 if sim_num % 2 == 0 else -2) * (1 if i % 2 == 0 else -1) for i in range(4)]
    tdcslist.frequency = frequency

    # INICIALIZAÇÃO DOS CÁTODOS
    for cathode_ch in selected_cathodes:
        cathode = tdcslist.add_electrode()
        cathode.channelnr = cathode_ch
        cathode.dimensions = [30]  # Define o diâmetro do eletrodo cátodo como 30 mm
        cathode.shape = 'ellipse'  # Define a forma do eletrodo cátodo como circular
        cathode.thickness = 5
        cathode.centre = random.choice(safe_positions)

    # INICIALIZAÇÃO DOS ÂNODOS
    for anode_ch in selected_anodes:
        anode = tdcslist.add_electrode()
        anode.channelnr = anode_ch
        anode.dimensions = [30]  # Define o diâmetro do eletrodo ânodo como 30 mm
        anode.shape = 'ellipse'  # Define a forma do eletrodo ânodo como circular
        anode.thickness = 5

        # DEFINA A POSIÇÃO DO ÂNODO DIFERENTE DA POSIÇÃO DO CÁTODO
        other_positions = [pos for pos in safe_positions if pos not in selected_cathodes]
        anode.centre = random.choice(other_positions)


# CLASSE PARA FECHAR AS JANELAS .msh
def close_all_msh_windows():
    while True:
        for window in gw.getAllTitles():
            if window.endswith(".msh"):
                window_to_close = gw.getWindowsWithTitle(window)
                if window_to_close:
                    window_to_close[0].close()
        time.sleep(1)  # Fecha em um segundo

# RODAR A SIMULAÇÃO
def run_simulation(s):
    run_simnibs(s)

# INICIA PARA RODAR AS SIMULAÇÕES
simulation_thread = threading.Thread(target=run_simulation, args=(s,))
simulation_thread.start()

# FECHA AS JANELAS .msh
window_closing_thread = threading.Thread(target=close_all_msh_windows)
window_closing_thread.start()

# COMANDO PARA AGUARDAR O PROCESSO
simulation_thread.join()

# AO TERMINAR OS PROCESSOS DA SIMULAÇÃO É FINALIZADO A THREAD
window_closing_thread.join()

**6 eletrodos**

In [ ]:
import random
import time
import threading
import pygetwindow as gw
from itertools import combinations
from simnibs import sim_struct, run_simnibs

# INICIALIZANDO A SESSÃO
s = sim_struct.SESSION()

# NOME DA MALHA DA CABEÇA
s.subpath = r'C:\Users\laboratorio\Documents\simnibs4_examples\m2m_ernie'

# PASTA DE SAÍDA
s.pathfem = 'tutorial6/'

# LISTA DE POSIÇÕES
safe_positions = ['AF3', 'AF4', 'AF7', 'AF8', 'AFz', 'C1', 'C2', 'C3', 'C4', 'C5', 'C6', 'CP1', 'CP2', 'CP3', 'CP4', 'CP5',
                   'CP6', 'CPz', 'Cz', 'F1', 'F10', 'F2', 'F3', 'F4', 'F5', 'F6', 'F7', 'F8', 'F9', 'FC1', 'FC2', 'FC3',
                   'FC4', 'FC5', 'FC6', 'FCz', 'FT10', 'FT7', 'FT8', 'FT9', 'Fp1', 'Fp2', 'Fpz', 'Fz', 'I1', 'I2', 'Iz',
                   'LPA', 'Nz', 'P1', 'P10', 'P2', 'P3', 'P4', 'P5', 'P6', 'P7', 'P8', 'P9', 'PO10', 'PO3', 'PO4', 'PO7',
                   'PO8', 'PO9', 'POz', 'Pz', 'RPA', 'T10', 'T7', 'T8', 'T9', 'TP10', 'TP7', 'TP8', 'TP9']

# GERANDO LISTA DE CANAIS DOS ELETRODOS PARA 6 ELETRODOS
electrode_channels_6 = [1, 2, 3, 4, 5, 6]
valid_electrode_combinations_6 = list(combinations(electrode_channels_6, 6))

# NÚMERO DE SIMULAÇÕES PARA 6 ELETRODOS
num_simulations_6 = 50

simulations_data = []

# LOOP DE SIMULAÇÃO PARA 6 ELETRODOS
for sim_num in range(num_simulations_6):

    # SELECIONE UMA COMBINAÇÃO DE ELETRODO
    electrodes = random.choice(valid_electrode_combinations_6)

    # SELECIONANDO 3 CÁTODOS E 3 ÂNODOS ALEATORIAMENTE
    selected_cathodes = random.sample(electrodes, 3)
    selected_anodes = random.sample([ch for ch in electrodes if ch not in selected_cathodes], 3)

    # INICIALIZAÇÃO DA SIMULAÇÃO tDCS
    tdcslist = s.add_tdcslist()

    # Adicione os parâmetros de corrente alternada
    frequency = 1  # Frequência em Hertz
    amplitude = 1e-3  # Amplitude da corrente em Ampères
    tdcslist.currents = [amplitude * (2 if sim_num % 2 == 0 else -2) * (1 if i % 2 == 0 else -1) for i in range(6)]
    tdcslist.frequency = frequency

    # INICIALIZAÇÃO DOS CÁTODOS
    for cathode_ch in selected_cathodes:
        cathode = tdcslist.add_electrode()
        cathode.channelnr = cathode_ch
        cathode.dimensions = [30]  # Define o diâmetro do eletrodo cátodo como 30 mm
        cathode.shape = 'ellipse'  # Define a forma do eletrodo cátodo como circular
        cathode.thickness = 5
        cathode.centre = random.choice(safe_positions)

    # INICIALIZAÇÃO DOS ÂNODOS
    for anode_ch in selected_anodes:
        anode = tdcslist.add_electrode()
        anode.channelnr = anode_ch
        anode.dimensions = [30]  # Define o diâmetro do eletrodo ânodo como 30 mm
        anode.shape = 'ellipse'  # Define a forma do eletrodo ânodo como circular
        anode.thickness = 5

        # DEFINA A POSIÇÃO DO ÂNODO DIFERENTE DA POSIÇÃO DO CÁTODO
        other_positions = [pos for pos in safe_positions if pos not in selected_cathodes]
        anode.centre = random.choice(other_positions)


# CLASSE PARA FECHAR AS JANELAS .msh
def close_all_msh_windows():
    while True:
        for window in gw.getAllTitles():
            if window.endswith(".msh"):
                window_to_close = gw.getWindowsWithTitle(window)
                if window_to_close:
                    window_to_close[0].close()
        time.sleep(1)  # Fecha em um segundo

# RODAR A SIMULAÇÃO
def run_simulation(s):
    run_simnibs(s)

# INICIA PARA RODAR AS SIMULAÇÕES
simulation_thread = threading.Thread(target=run_simulation, args=(s,))
simulation_thread.start()

# FECHA AS JANELAS .msh
window_closing_thread = threading.Thread(target=close_all_msh_windows)
window_closing_thread.start()

# COMANDO PARA AGUARDAR O PROCESSO
simulation_thread.join()

# AO TERMINAR OS PROCESSOS DA SIMULAÇÃO É FINALIZADO A THREAD
window_closing_thread.join()

**8 eletrodos**

In [ ]:
import random
import time
import threading
import pygetwindow as gw
from itertools import combinations
from simnibs import sim_struct, run_simnibs

# INICIALIZANDO A SESSÃO
s = sim_struct.SESSION()

# NOME DA MALHA DA CABEÇA
s.subpath = r'C:\Users\laboratorio\Documents\simnibs4_examples\m2m_ernie'

# PASTA DE SAÍDA
s.pathfem = 'tutorial8/'

# LISTA DE POSIÇÕES
safe_positions = ['AF3', 'AF4', 'AF7', 'AF8', 'AFz', 'C1', 'C2', 'C3', 'C4', 'C5', 'C6', 'CP1', 'CP2', 'CP3', 'CP4', 'CP5',
                   'CP6', 'CPz', 'Cz', 'F1', 'F10', 'F2', 'F3', 'F4', 'F5', 'F6', 'F7', 'F8', 'F9', 'FC1', 'FC2', 'FC3',
                   'FC4', 'FC5', 'FC6', 'FCz', 'FT10', 'FT7', 'FT8', 'FT9', 'Fp1', 'Fp2', 'Fpz', 'Fz', 'I1', 'I2', 'Iz',
                   'LPA', 'Nz', 'P1', 'P10', 'P2', 'P3', 'P4', 'P5', 'P6', 'P7', 'P8', 'P9', 'PO10', 'PO3', 'PO4', 'PO7',
                   'PO8', 'PO9', 'POz', 'Pz', 'RPA', 'T10', 'T7', 'T8', 'T9', 'TP10', 'TP7', 'TP8', 'TP9']

# GERANDO LISTA DE CANAIS DOS ELETRODOS PARA 8 ELETRODOS
electrode_channels_8 = [1, 2, 3, 4, 5, 6, 7, 8]
valid_electrode_combinations_8 = list(combinations(electrode_channels_8, 8))

# NÚMERO DE SIMULAÇÕES PARA 8 ELETRODOS
num_simulations_8 = 50

simulations_data = []

# LOOP DE SIMULAÇÃO PARA 8 ELETRODOS
for sim_num in range(num_simulations_8):

    # SELECIONE UMA COMBINAÇÃO DE ELETRODO
    electrodes = random.choice(valid_electrode_combinations_8)

    # SELECIONANDO 4 CÁTODOS E 4 ÂNODOS ALEATORIAMENTE
    selected_cathodes = random.sample(electrodes, 4)
    selected_anodes = random.sample([ch for ch in electrodes if ch not in selected_cathodes], 4)

    # INICIALIZAÇÃO DA SIMULAÇÃO tDCS
    tdcslist = s.add_tdcslist()

    # Adicione os parâmetros de corrente alternada
    frequency = 1  # Frequência em Hertz
    amplitude = 1e-3  # Amplitude da corrente em Ampères
    tdcslist.currents = [amplitude * (2 if sim_num % 2 == 0 else -2) * (1 if i % 2 == 0 else -1) for i in range(8)]
    tdcslist.frequency = frequency

    # INICIALIZAÇÃO DOS CÁTODOS
    for cathode_ch in selected_cathodes:
        cathode = tdcslist.add_electrode()
        cathode.channelnr = cathode_ch
        cathode.dimensions = [30]  # Define o diâmetro do eletrodo cátodo como 30 mm
        cathode.shape = 'ellipse'  # Define a forma do eletrodo cátodo como circular
        cathode.thickness = 5
        cathode.centre = random.choice(safe_positions)

    # INICIALIZAÇÃO DOS ÂNODOS
    for anode_ch in selected_anodes:
        anode = tdcslist.add_electrode()
        anode.channelnr = anode_ch
        anode.dimensions = [30]  # Define o diâmetro do eletrodo ânodo como 30 mm
        anode.shape = 'ellipse'  # Define a forma do eletrodo ânodo como circular
        anode.thickness = 5

        # DEFINA A POSIÇÃO DO ÂNODO DIFERENTE DA POSIÇÃO DO CÁTODO
        other_positions = [pos for pos in safe_positions if pos not in selected_cathodes]
        anode.centre = random.choice(other_positions)


# CLASSE PARA FECHAR AS JANELAS .msh
def close_all_msh_windows():
    while True:
        for window in gw.getAllTitles():
            if window.endswith(".msh"):
                window_to_close = gw.getWindowsWithTitle(window)
                if window_to_close:
                    window_to_close[0].close()
        time.sleep(1)  # Fecha em um segundo

# RODAR A SIMULAÇÃO
def run_simulation(s):
    run_simnibs(s)

# INICIA PARA RODAR AS SIMULAÇÕES
simulation_thread = threading.Thread(target=run_simulation, args=(s,))
simulation_thread.start()

# FECHA AS JANELAS .msh
window_closing_thread = threading.Thread(target=close_all_msh_windows)
window_closing_thread.start()

# COMANDO PARA AGUARDAR O PROCESSO
simulation_thread.join()

# AO TERMINAR OS PROCESSOS DA SIMULAÇÃO É FINALIZADO A THREAD
window_closing_thread.join()